In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

hdb_link = "https://services2.hdb.gov.sg/webapp/BB33RTIS/BB33SComparator"
hdb_title = "Resale Flat Prices"
user_profile_title = "userProfileB"
flat_type_title = "FLAT_TYPE"
flat_type_val = Keys.NUMPAD4
town_title = "NME_NEWTOWN"
town_val = None
# town_val = "a"
street_title = "NME_STREET"
# street_val = None
street_val = "Ang Mo Kio Ave 3"
date_range_title = "dteRange"
submit_btn_id = "btnSubmit"

params = {"hdb_link":hdb_link,
          "hdb_title":hdb_title,
          "user_profile_title":user_profile_title,
          "flat_type_title":flat_type_title,
          "flat_type_val":flat_type_val,
          "town_title":town_title,
          "town_val":town_val,
          "street_title":street_title,
          "street_val":street_val,
          "date_range_title":date_range_title,
          "submit_btn_id":submit_btn_id}

In [2]:
# start the driver, returns a webdriver chrome object
def start_driver() -> webdriver.Chrome:
    driver = webdriver.Chrome()
    driver.get(hdb_link)
    assert hdb_title in driver.title
    return driver

In [3]:
# runs the query in the website with HDB town 
def run_query_hdb(driver: webdriver.Chrome, params:dict):
    print("running hdb")
    hdb_town = driver.find_element(By.NAME, params["town_title"])
    hdb_town.send_keys(params["town_val"])


# runs the query in the website with Street Name
def run_query_street(driver: webdriver.Chrome, params:dict):
    print("running street")
    street_field = driver.find_element(By.NAME , params["street_title"])
    street_field.send_keys(params["street_val"])

    
# runs the base query that is similar to both hdb or street query
def run_base_query(driver: webdriver.Chrome, params:dict):
    radio = driver.find_element(By.ID, params["user_profile_title"])
    radio.click()
    flat_type = driver.find_element(By.NAME, params["flat_type_title"])
    flat_type.send_keys(params["flat_type_val"])

    date_range_field = driver.find_element(By.ID , params["date_range_title"])
    date_range_field.send_keys("last 12")

# master query function
# returns all records in chronological order
def run_query(driver: webdriver.Chrome, params: dict):
    if params["street_val"]:
        run_query_street(driver, params)
    else:
        run_query_hdb(driver, params)
    run_base_query(driver, params)
    submit_btn = driver.find_element(By.ID, params["submit_btn_id"])
    submit_btn.click()

In [4]:
# Runs the query twice which for some reason will then only be able to generate the results
def run_query_success(driver: webdriver.Chrome, params:dict):
    run_query(driver, params)
    run_query(driver, params)

In [5]:
driver = start_driver()
run_query_success(driver, params)


running street
running street


In [6]:
headers = [
    "Block /Nearby Amenities",
    "Street Name",
    "Storey",
    "Floor Area (sqm)" , 
    "Flat Model",
    "Lease Commence Date",
    "Remaining Lease",
    "Resale Price",
    "Resale Registration Date"
    ]

In [7]:
headers = driver.find_elements(By.XPATH, '//tr[@class="svcTableHeader"]')

In [8]:
headers[0].text

'HDB Town\nBlock /\nNearby\nAmenities\nStorey\nFloor Area (sqm) /\nFlat Model\nLease Commence Date\nRemaining Lease \nResale\nPrice\nResale\nRegistration\nDate'

In [9]:
addresses = driver.find_elements(By.XPATH, '//tr[@height="30"]')
# we only need half of them
addresses = addresses[:int(len(addresses)/2)]

In [10]:
addresses_tr = addresses[1].find_elements(By.XPATH,".//child::td")

In [11]:
len(addresses)

28

In [12]:
len(addresses_tr)

8

In [13]:
for i, address in enumerate(addresses_tr):
    print("index is :" + str(i) + " value  " + address.text)

index is :0 value  
index is :1 value  
index is :2 value  
index is :3 value  
index is :4 value  
index is :5 value  
index is :6 value  
index is :7 value  


In [14]:
data = []

In [15]:
counter=0
for address in addresses:
    if counter >3:
        break
    tr = address.find_elements(By.XPATH,".//child::td")
    # for tr_data in tr:
    #     print(tr_data.text)
    
    block = tr[0].text
    print(tr[0])
    street_name = tr[1].text
    storey = tr[2].text
    floor_area,flat_model = tr[3].text.split("\n")
    lease_commence = tr[4].text
    remaining_lease = tr[5].text.split("\n")[0]
    print(remaining_lease)
    resale_price = tr[6].text
    resale_date = tr[7].text
    data.append([block, street_name, storey, floor_area, flat_model, lease_commence, remaining_lease, resale_price, resale_date])
    counter +=1


<selenium.webdriver.remote.webelement.WebElement (session="f76c8f1a7180c5cc11557b9506d786d5", element="B4B81B857C4F5BA51EA9879FD54D95D8_element_301")>


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
data

[['310A',
  ' Ang Mo Kio Ave 1',
  '04 to 06',
  '94.00',
  'Model A',
  '2012',
  '87 years\n11 months',
  '$770,000.00',
  'Nov 2023'],
 ['411',
  ' Ang Mo Kio Ave 10',
  '07 to 09',
  '92.00',
  'New Generation',
  '1979',
  '54 years\n11 months',
  '$480,000.00',
  'Nov 2023'],
 ['354',
  ' Ang Mo Kio St 32',
  '13 to 15',
  '95.00',
  'Model A',
  '2001',
  '76 years\n11 months',
  '$770,000.00',
  'Nov 2023'],
 ['455A',
  ' Ang Mo Kio St 44',
  '04 to 06',
  '93.00',
  'Model A',
  '2018',
  '93 years\n5 months',
  '$765,000.00',
  'Nov 2023'],
 ['310A',
  ' Ang Mo Kio Ave 1',
  '04 to 06',
  '94.00',
  'Model A',
  '2012',
  '87 years',
  '$770,000.00',
  'Nov 2023'],
 ['411',
  ' Ang Mo Kio Ave 10',
  '07 to 09',
  '92.00',
  'New Generation',
  '1979',
  '54 years',
  '$480,000.00',
  'Nov 2023'],
 ['354',
  ' Ang Mo Kio St 32',
  '13 to 15',
  '95.00',
  'Model A',
  '2001',
  '76 years',
  '$770,000.00',
  'Nov 2023'],
 ['455A',
  ' Ang Mo Kio St 44',
  '04 to 06',
  '93.00'